# [MLP_base] Standardized Baseline

이 노트북은 모든 모델 성능 비교의 **기준점(Reference Point)**이 되는 Baseline 모델입니다.  
기교를 부리지 않은 **"가장 단순한 구조"**로, 데이터가 가진 기본적인 예측 능력을 평가합니다.

### 📌 핵심 구성 (Configuration)
1.  **Simple Architecture**: Input, Hidden(2층), Output으로만 구성된 **BasicMLP**를 사용합니다. (No DropOut, No BatchNorm)
2.  **Standard Pipeline**: `MLP_enhance`와 동일한 데이터(Anchors 제거, Raw Distribution) 규격을 따릅니다. --> **공정한 비교 보장**
3.  **Fast Execution**: Epochs 10회 설정을 통해 빠르게 기준 점수를 산출합니다.

---

## 1. 라이브러리 임포트 및 모델 정의 (Libraries & Model)

- **라이브러리**: PyTorch, Pandas 등 필수 도구를 임포트합니다.
- **BasicMLP**: 3-Layer(Input-Hidden-Output)로 구성된 가장 기본적인 신경망 모델을 정의합니다.
- **특징**: Dropout이나 Batch Normalization 없이 순수한 Fully Connected Layer로만 구성됩니다.

In [ ]:
import sys
import os
import pathlib

sys.path.append(os.path.abspath("../../"))

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import json
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
from IPython.display import display, Markdown

# [MPS Acceleration]
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"🚀 Using Device: {device}")

# [Model Import] BasicMLP (From centralized definitions)
from models.model_definitions import BasicMLP
print("✅ BasicMLP Model Imported.")


## 2. 데이터 로드 및 전처리 (Data Loading)

- **Clean Data Pipeline**: `MLP_enhance`와 동일하게 정제된 파케이 데이터를 사용합니다.
- **Type Casting**: ID 컬럼을 문자열로 변환하여 매칭 정확도를 보장합니다.
- **Inner Join**: Feature와 Label이 모두 존재하는 데이터만 병합합니다.

In [ ]:
print('📂 Loading Data...')
base_path = "/Users/gimdabin/SKN23-2nd-3Team/data/processed/"

# Load Parquet Files
features = pd.read_parquet(base_path + "features_ml_clean.parquet")
labels = pd.read_parquet(base_path + "labels.parquet")

# Type Casting
features['user_id'] = features['user_id'].astype(str)
labels['user_id'] = labels['user_id'].astype(str)

# Merge (Inner Join)
data = features.merge(labels, on=['user_id', 'anchor_time'], how='inner')
data['target'] = (data['label'] == 'm2').astype(int)

# Features & Target Split
feature_cols = [c for c in features.columns if c not in ['user_id', 'anchor_time']]
X = data[feature_cols].copy().fillna(0)
y = data['target'].values

print(f"✅ Data Loaded. Shape: {data.shape}")


## 3. 데이터 분할 (Time-based Split)

- **Time-based Split**: `split` 컬럼을 사용하여 학습/검증/테스트셋을 나눕니다.
- **Consistency**: 다른 모델들과 동일한 데이터 셋을 사용하여 성능 비교의 공정성을 확보합니다.

In [ ]:
assert "split" in data.columns, "Missing 'split' column!"

split = data["split"].astype(str).str.lower().values
train_mask = split == "train"
val_mask   = np.isin(split, ["val", "valid", "validation"])
test_mask  = split == "test"

X_train, y_train = X.loc[train_mask].values, y[train_mask]
X_val, y_val     = X.loc[val_mask].values, y[val_mask]
X_test, y_test   = X.loc[test_mask].values, y[test_mask]

print(f"🔹 Train: {len(X_train)}, Val: {len(X_val)}, Test: {len(X_test)}")
print(f"📊 Train Base Rate: {y_train.mean():.4f}")
print(f"📊 Val   Base Rate: {y_val.mean():.4f}")
print(f"📊 Test  Base Rate: {y_test.mean():.4f}")


## 4. 데이터 스케일링 (Preprocessing)

- **Standard Scaling**: Train Set의 통계치를 기준으로 전체 데이터를 정규화합니다.
- **No SMOTE**: Baseline은 데이터를 왜곡하지 않고 있는 그대로(Raw) 학습하여 실제 성능의 하한선을 측정합니다.

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

BATCH_SIZE = 256
train_loader = DataLoader(TensorDataset(torch.FloatTensor(X_train_scaled), torch.FloatTensor(y_train)), batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(TensorDataset(torch.FloatTensor(X_val_scaled), torch.FloatTensor(y_val)), batch_size=BATCH_SIZE)
test_loader = DataLoader(TensorDataset(torch.FloatTensor(X_test_scaled), torch.FloatTensor(y_test)), batch_size=BATCH_SIZE)

print("✅ Preprocessing Compelete (Raw Distribution).")


## 5. 하이퍼파라미터 설정 (Fast Config)

- **Hidden Dim**: 128 (가벼운 모델)
- **Epochs**: 10 (빠른 벤치마킹)
- **Optimzier**: Adam (기본 최적화 알고리즘)

In [ ]:
best_params = {
    'lr': 0.001,
    'weight_decay': 0.00001,
    'hidden_dim': 128,
    'epochs': 10,
    'optimizer': 'Adam'
}
print(f"⚙️ Config Loaded: {best_params}")


## 6. 모델 학습 (Training Loop)

- **Training**: `BasicMLP` 모델을 학습시킵니다.
- **Validation**: 매 Epoch마다 성능을 체크하지만, Baseline이므로 별도의 복잡한 Early Stopping은 적용하지 않습니다.

In [ ]:
model = BasicMLP(X.shape[1], hidden_dim=best_params['hidden_dim']).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=best_params['lr'], weight_decay=best_params['weight_decay'])

print("🚀 Starting Baseline Training...")

for epoch in range(best_params['epochs']):
    model.train()
    epoch_loss = 0
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs).squeeze()
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        
    # Validation Log
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for inputs, targets in val_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs).squeeze()
            val_loss += criterion(outputs, targets).item()
            
    print(f'Epoch {epoch+1:02d}/{best_params["epochs"]}: Train Loss {epoch_loss/len(train_loader):.4f}, Val Loss {val_loss/len(val_loader):.4f}')


## 7. 모델 평가 (Evaluation)

- **Comparison**: 이 모델의 성능 점수는 다른 고급 모델들(`Enhance`, `Advanced`)이 반드시 넘어야 할 기준선입니다.
- **Metrics**: ROC-AUC, Precision, Lift 등을 측정합니다.

In [ ]:
from app.utils.metrics import evaluate_churn_metrics

model.eval()
all_probs = []
all_true = []

with torch.no_grad():
    for inputs, targets in test_loader:
        inputs = inputs.to(device)
        outputs = model(inputs).squeeze()
        probs = torch.sigmoid(outputs)
        all_probs.extend(probs.cpu().numpy())
        all_true.extend(targets.numpy())

metrics_result = evaluate_churn_metrics(
    y_true=np.array(all_true),
    y_prob=np.array(all_probs)
)

# Ranking 분리
if 'ranking' in metrics_result:
    ranking_list = metrics_result.pop('ranking')
    top_k_df = pd.DataFrame(ranking_list)
else:
    top_k_df = pd.DataFrame()

metrics = metrics_result

display(Markdown("### 📊 Baseline Performance"))
display(pd.DataFrame(list(metrics.items()), columns=['KPI', 'Value']))
display(Markdown("### 📈 Top K% Ranking"))
display(top_k_df)


## 8. 결과 저장 (Artifact Saving)

- **Artifacts**: 모델 평가 결과를 표준화된 JSON 포맷으로 저장합니다.
- **Path**: `models/eval/dlmlp_base/`

In [ ]:
import json
import pathlib

# [Correct Path] dlmlp_base
MODEL_ID = "dlmlp_base"
EVAL_DIR = pathlib.Path(f"../../models/eval/{MODEL_ID}")
EVAL_DIR.mkdir(parents=True, exist_ok=True)

# Model Card
model_card = {
    "model_id": MODEL_ID,
    "display_name": "MLP Base (Baseline)",
    "category": "DL",
    "split": "test"
}
with open(EVAL_DIR / "model_card.json", "w") as f:
    json.dump(model_card, f, indent=2, ensure_ascii=False)

# PR Metrics
pr_metrics = {
    "model_id": MODEL_ID,
    "split": "test",
    "pr_auc": float(metrics.get("PR-AUC (Average Precision)", 0.0))
}
with open(EVAL_DIR / "pr_metrics.json", "w") as f:
    json.dump(pr_metrics, f, indent=2, ensure_ascii=False)

# Top K Metrics
current_base_rate = float(np.mean(all_true)) if 'all_true' in locals() else 0.0
metrics_by_k = []
if not top_k_df.empty:
    for _, row in top_k_df.iterrows():
        k_str = str(row.get('Top_K', '0')).replace('%', '')
        try: k_val = int(k_str)
        except: k_val = 0
        metrics_by_k.append({
            "k_pct": k_val,
            "precision_at_k": float(row.get('Precision', 0.0)),
            "recall_at_k": float(row.get('Recall', 0.0)),
            "lift_at_k": float(row.get('Lift', 0.0))
        })

topk_output = {
    "model_id": MODEL_ID,
    "split": "test",
    "base_rate": current_base_rate,
    "metrics_by_k": metrics_by_k
}
with open(EVAL_DIR / "topk_metrics.json", "w") as f:
    json.dump(topk_output, f, indent=2, ensure_ascii=False)

print(f"✅ All artifacts saved to {EVAL_DIR} (LG Style Format)")
